## Temporal Difference: N-Step Sarsa

In [1]:
# 获取一个格子的状态
def get_state(row, col):
  if row != 3:
    return 'ground'
  if row == 3 and col == 0:
    return 'ground'
  if row == 3 and col == 11:
    return 'terminal'
  
  return 'trap'

get_state(0, 0)

'ground'

In [2]:
# 在一个格子里做一个动作
def move(row, col, action):
  # 如果当前已经在陷阱或者重点，则不能执行任何动作
  if get_state(row, col) in ['trap', 'terminal']:
    return row, col, 0
  # ↑
  if action == 0:
    row -= 1
  # ↓
  if action == 1:
    row += 1
  # ←
  if action == 2:
    col -= 1
  # →
  if action == 3:
    col += 1
    
  # 不允许走到地图外面
  row = max(0, row)
  row = min(3, row)
  col = max(0, col)
  col = min(11, col)
  
  # 陷阱-100，否则都是-1
  reward = -1
  if get_state(row, col) == 'trap':
    reward = -100
    
  return row, col, reward
  
move(0, 0, 3)



(0, 1, -1)

In [5]:
import numpy as np

# 初始化每个格子里采取每个动作的分数，
# 均为0， 因为没有先验知识
Q = np.zeros([4, 12, 4]) # 全局变量

## new code begin here
# 初始化3个list, 用来存储状态，动作，反馈的历史数据
state_list = []
action_list = []
reward_list = []
## new code end here

Q.shape

(4, 12, 4)

In [6]:
import random

# 根据状态选择动作，e-greedy
def get_action(row, col):
  # 小概率随机选择
  if random.random() < 0.1: # epsilon
    return random.choice(range(4))
  #
  # 否则选择分数最高的动作
  return Q[row, col].argmax()

get_action(0, 0)

0

In [8]:
# new code begin here
# 获取5个时间步的分数
def get_update_list(next_row, next_col, next_action):
  # 初始化最后一个state和最后一个action对应的target分数
  target = Q[next_row, next_col, next_action]

  # 计算每一步的target
  #每一步的tagret等于下一步的tagret*0.9，再加上本步的reward
  #时间从后往前回溯，越以前的tagret会累加的信息越多
  #[4, 3, 2, 1, 0]
  target_list = []
  for i in reversed(range(5)):
    target = 0.9 * target + reward_list[i]
    target_list.append(target)
    
  # 回正时间顺序
  target_list = list(reversed(target_list))
  
  # 计算每一步value
  value_list = []
  for i in range(5):
    row, col = state_list[i]
    action = action_list[i]
    value_list.append(Q[row, col, action])
  
  # 计算每一步的更新量
  update_list = []
  for i in range(5):
    #根据时序差分算法,当前state,action的分数 = 下一个state,action的分数*gamma + reward
    #此处是求两者的差,越接近0越好
    update = target_list[i] - value_list[i]
    
    # 学习率 0.1
    update *= 0.1
    
    update_list.append(update)
  
  # 更新list中每个状态的动作的分数
  return update_list

In [9]:
# 训练 training
def train():
  for epoch in range(1500):
    # init curr loc
    row = random.choice(range(4))
    col = 0
    
    # init first action 
    action = get_action(row, col)
    
    # 计算反馈和，应当逐渐减小
    reward_sum = 0
    
    # new code begin here
    # 初始化3个列表 
    state_list.clear()
    action_list.clear()
    reward_list.clear()
    # new code end here 
    
    # 循环直到终止
    while get_state(row, col) not in ['terminal', 'trap']:
      
      # 执行动作
      next_row, next_col, reward = move(row, col, action)
      reward_sum += reward
      
      # 求新位置的动作
      next_action = get_action(next_row, next_col)
      
      # new code begin here ------------
      # 记录历史数据
      state_list.append([row, col])
      action_list.append(action)
      reward_list.append(reward)
      
      # 累积5步后更新分数
      if len(state_list) == 5:
        # 计算分数
        update_list = get_update_list(next_row, next_col, next_action)

        # 只更新第一步的分数
        row, col = state_list[0]
        action = action_list[0]
        update = update_list[0]
        
        Q[row, col, action] += update
        
        # 移除第一步，保持列表长度
        state_list.pop(0)
        action_list.pop(0)
        reward_list.pop(0)
      
      # 更新当前位置
      row = next_row
      col = next_col
      action = next_action
      
    # 走到终点以后，更新剩下的步数update
    for i in range(len(state_list)):
      row, col = state_list[i]
      action = action_list[i]
      update = update_list[i]
      Q[row, col, action] += update
      
    if epoch % 100 == 0:
      print(epoch, reward_sum)
      # new code end here   ------------
    
train()

0 -199
100 -23
200 -20
300 -19
400 -19
500 -15
600 -52
700 -19
800 -24
900 -19
1000 -15
1100 -19
1200 -24
1300 -15
1400 -22


In [10]:
#打印游戏，方便测试
def show(row, col, action):
  graph = [
      '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□',
      '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□',
      '□', '□', '□', '□', '□', '□', '□', '□', '□', '○', '○', '○', '○', '○',
      '○', '○', '○', '○', '○', '❤'
  ]

  action = {0: '↑', 1: '↓', 2: '←', 3: '→'}[action]

  graph[row * 12 + col] = action

  graph = ''.join(graph)

  for i in range(0, 4 * 12, 12):
    print(graph[i:i + 12])


show(1, 1, 0)

□□□□□□□□□□□□
□↑□□□□□□□□□□
□□□□□□□□□□□□
□○○○○○○○○○○❤


In [11]:
from IPython import display
import time

def test():
  # 起点
  row = random.choice(range(4))
  col = 0
  
  # 最多玩N = 200步
  for _ in range(200):
    # 获取当前状态，终止态则终止
    if get_state(row, col) in ['trap', 'terminal']:
      break
    
    # 选择最优动作
    action = Q[row, col].argmax()
    
    # 打印此动作
    display.clear_output(wait=True)
    time.sleep(0.1)
    show(row, col, action)
    
    # 执行动作
    row, col, reward = move(row, col, action)

test()
    

□□□□□□□□□□□□
□□□□□□□□□□□□
□□□□□□□□□□□↓
□○○○○○○○○○○❤


In [12]:
# 打印所有格子上的动作倾向
for row in range(4):
  line = ''
  for col in range(12):
    action = Q[row, col].argmax()
    action = {0: '↑', 1: '↓', 2: '←', 3: '→'}[action]
    line += action
    
  print(line)

→→→→→→→→→→→↓
↑↑↑↑←→↑↑↑↑←↓
↑←↑→↑←↑→↑←→↓
↑↑↑↑↑↑↑↑↑↑↑↑
